<a href="https://colab.research.google.com/github/chetan-957/Data-Management-sp2025/blob/main/ps0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exploring the Relationship Between Economic Development & Food Security**

### Selecting the Datasets (Why These Data?)
For this project, we are using two key datasets to explore the relationship between economic development & food security across countries.

### 📊 Datasets Used:

✅ FAOSTAT Food Balance Sheets (from FAO)
link: https://www.fao.org/faostat/en/#data/FBS

Covers food supply, production, imports/exports, and nutritional indicators.
Tracks food availability & consumption trends from 2010 to 2023.
Key variables: Food supply (kcal/capita), protein & fat supply, food losses, stock variation.


✅ World Bank WDI (World Development Indicators)
link: https://databank.worldbank.org/source/world-development-indicators#

Covers economic & trade data for all countries from 1980-2023.
Key variables: GDP per capita, GNI per capita, Trade as % of GDP, Gini Index (income inequality).
📌 Why These Data?
These datasets allow us to analyze global food security patterns while linking them to economic growth & inequality. This helps answer critical questions like:
1️⃣ Does GDP per capita correlate with better food security?
2️⃣ Do trade policies affect food supply fluctuations?
3️⃣ Is income inequality (Gini Index) linked to undernourishment?
4️⃣ How have food security trends evolved over time?



### **1️⃣ Load & Inspect Datasets**

In [21]:
import gdown
import pandas as pd

# Google Drive file ID
file_id = "1QB05L0xO9XE6l-U5NQNrAkWsUPxOZpF_"

# Direct download URL
download_url = f"https://drive.google.com/uc?id={file_id}"

# Define output file name
output_file = "FoodBalanceSheets_E_All_Data.csv"

# Download the file
gdown.download(download_url, output_file, quiet=False)

# Load CSV
faostat_df = pd.read_csv(output_file, encoding="latin1")

# Display sample data
faostat_df.head()


Downloading...
From (original): https://drive.google.com/uc?id=1QB05L0xO9XE6l-U5NQNrAkWsUPxOZpF_
From (redirected): https://drive.google.com/uc?id=1QB05L0xO9XE6l-U5NQNrAkWsUPxOZpF_&confirm=t&uuid=31d09abd-a96e-4be4-bb09-dbe1a2bc8050
To: /content/FoodBalanceSheets_E_All_Data.csv
100%|██████████| 119M/119M [00:00<00:00, 185MB/s]


,Area Code,Area Code (M49),Area,Item Code,Item Code (FBS),Item,Element Code,Element,Unit,Y2010,...,Y2019N,Y2020,Y2020F,Y2020N,Y2021,Y2021F,Y2021N,Y2022,Y2022F,Y2022N
0,2,'004,Afghanistan,2501,'S2501,Population,511,Total Population - Both sexes,1000 No,28189.67,...,NaN,38972.23,X,NaN,40099.46,X,NaN,41128.77,X,NaN
1,2,'004,Afghanistan,2901,'S2901,Grand Total,664,Food supply (kcal/capita/day),kcal/cap/d,2208.00,...,NaN,2266.00,E,NaN,2240.00,E,NaN,2243.00,E,NaN
2,2,'004,Afghanistan,2901,'S2901,Grand Total,661,Food supply (kcal),million Kcal,22714261.33,...,NaN,32236668.33,E,NaN,32780360.22,E,NaN,33679315.09,E,NaN
3,2,'004,Afghanistan,2901,'S2901,Grand Total,674,Protein supply quantity (g/capita/day),g/cap/d,65.76,...,NaN,63.37,E,NaN,61.22,E,NaN,61.03,E,NaN
4,2,'004,Afghanistan,2901,'S2901,Grand Total,671,Protein supply quantity (t),t,676666.08,...,NaN,901465.91,E,NaN,895962.82,E,NaN,916131.81,E,NaN


In [22]:
# GitHub Raw File Link
github_url = "https://raw.githubusercontent.com/chetan-957/Data-Management-sp2025/main/P_Data_Extract_From_World_Development_Indicators.xlsx"

# Load WDI Excel file directly from GitHub
wdi_df = pd.read_excel(github_url, sheet_name="Data")

# Display sample data
wdi_df.head()


,Country Name,Country Code,Series Name,Series Code,1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,..,..,..,..,..,..,...,625.054942,565.56973,522.082216,525.469771,491.337221,496.602504,510.787063,356.496214,357.261153,415.707417
1,Afghanistan,AFG,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,..,..,..,..,..,..,...,640,590,560,530,520,520,490,380,370,380
2,Afghanistan,AFG,Gini index,SI.POV.GINI,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
3,Afghanistan,AFG,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,..,..,..,..,..,..,...,..,..,54.5,..,..,..,..,..,..,..
4,Afghanistan,AFG,Poverty headcount ratio at $2.15 a day (2017 P...,SI.POV.DDAY,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
